In [ ]:
import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 
src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset_utils import create_nuscenes_odometry_dataset
from dataset.filters.filter_list import FilterList
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose
import scipy
from scipy.spatial.distance import cdist
from normalized_cut import normalized_cut
from ncuts_utils import ncuts_chunk,kDTree_1NN_feature_reprojection_colors, get_merge_pcds
from dataset_utils import * 
from point_cloud_utils import get_pcd, transform_pcd, kDTree_1NN_feature_reprojection, remove_isolated_points, get_subpcd, get_statistical_inlier_indices, merge_chunks_unite_instances, merge_unite_gt, remove_semantics, merge_chunks_unite_instances2
from aggregate_pointcloud import aggregate_pointcloud
from visualization_utils import generate_random_colors, color_pcd_by_labels,generate_random_colors_map
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch, dinov2_mean, get_indices_feature_reprojection
from metrics.metrics_class import Metrics
import shutil
from tqdm import tqdm 

In [ ]:
DATASET_PATH = '/media/cedric/Datasets1/nuScenes_mini_v2/nuScenes'



dist_threshold = 5 #moving object filter threshold 
#dataset_type = 'v1.0-trainval'
dataset_type = 'v1.0-mini'
if dataset_type == 'v1.0-trainval' :  ##this script can be used for full eval with path change 
        DATASET_PATH = '/media/cedric/Datasets1/nuScenes_train'

minor_voxel_size = 0.05
major_voxel_size = 0.35
chunk_size = np.array([25, 25, 25]) #meters
overlap = 3 #meters
ground_segmentation_method = 'patchwork' 
NCUT_ground = False 


out_folder = 'pcd_preprocessed_nuscenes/' + dataset_type + '/'
if os.path.exists(out_folder) == False : 
        os.makedirs(out_folder)

out_nuscenes_pred = f'{out_folder}out_nuscenes_ncuts/'
out_nuscenes_instances = f'{out_folder}out_nuscenes_instance/'
map_out_instances = f'{out_folder}out_nuscenes_instance/maps/'
map_out_pred = f'{out_folder}out_nuscenes_ncuts/maps/'

if os.path.exists(map_out_instances) == False : 
                os.makedirs(map_out_instances)
                
if os.path.exists(map_out_pred) == False : 
                os.makedirs(map_out_pred)

alpha = 0.0
theta = 0.1
beta = 0.0
gamma = 0.0
proximity_threshold = 1.0
T = 0.001
seqs = list(range(0,10)) ## the mini dataset has 10 seqs max 

all_instances = []



Here we define the dataset depending on nuscenes sequence!

In [10]:



colors = generate_random_colors_map(600,0)
for SEQUENCE_NUM in tqdm(seqs) : 
        print('current sequence',SEQUENCE_NUM)
        dataset = create_nuscenes_odometry_dataset(DATASET_PATH,SEQUENCE_NUM,ncuts_mode=True, sam_folder_name="SAM", 
                        dinov2_folder_name="Dinov2",dist_threshold=dist_threshold,dataset_type=dataset_type)
        
        ind_start = 0
        ind_end = len(dataset)  
                        
        #if os.path.exists(f'{out_folder}all_poses_' + str(SEQUENCE_NUM) + '_' + str(0) + '.npz') == False:
        process_and_save_point_clouds(dataset,ind_start,ind_end,minor_voxel_size=minor_voxel_size,
                                major_voxel_size=major_voxel_size,icp=False,
                                out_folder=out_folder,sequence_num=SEQUENCE_NUM,
                                ground_segmentation_method=ground_segmentation_method)
        
        
        #if os.path.exists(f'{out_folder}pcd_ground_minor' + str(SEQUENCE_NUM) + '.pcd') == False:
        pcd_ground_minor, pcd_nonground_minor,\
                all_poses, T_pcd, first_position,labels = load_and_downsample_point_clouds(out_folder,SEQUENCE_NUM,minor_voxel_size,\
                                                                        ground_mode=ground_segmentation_method)

        o3d.io.write_point_cloud(f'{out_folder}pcd_ground_minor.pcd', pcd_ground_minor, write_ascii=False, compressed=False, print_progress=False)
        o3d.io.write_point_cloud(f'{out_folder}pcd_nonground_minor.pcd', pcd_nonground_minor, write_ascii=False, compressed=False, print_progress=False)
        np.savez(f'{out_folder}nuscenes_labels_preprocessed.npz',
                                                instance_nonground= labels['instance_nonground'],
                                                instance_ground= labels['instance_ground'],
                                                seg_ground = labels['seg_ground'],
                                                seg_nonground= labels['seg_nonground']
                                                )
        
        
        pcd_ground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_ground_minor.pcd')
        pcd_nonground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_nonground_minor.pcd')
        
        nuscenes_labels_orig = {}
        with np.load(f'{out_folder}nuscenes_labels_preprocessed.npz') as data :
                nuscenes_labels_orig['instance_ground'] = data['instance_ground']
                nuscenes_labels_orig['instance_nonground'] = data['instance_nonground']
                nuscenes_labels_orig['seg_nonground'] = data['seg_nonground']
                nuscenes_labels_orig['seg_ground'] = data['seg_ground']
        
                
        
        with np.load(f'{out_folder}all_poses_{SEQUENCE_NUM}_0.npz') as data:
                all_poses = data['all_poses']
                T_pcd = data['T_pcd']
                first_position = T_pcd[:3, 3]
        
        
        pcd_new = o3d.geometry.PointCloud()
        pcd_new.points = o3d.utility.Vector3dVector(np.asarray(pcd_nonground_minor.points))
        
        map_labelled = color_pcd_by_labels(pcd_new,\
                        nuscenes_labels_orig['instance_nonground'].reshape(-1,1))
        
        #o3d.visualization.draw_geometries([map_labelled])
        
        poses, positions, \
        sampled_indices_local, sampled_indices_global = subsample_and_extract_positions(all_poses,ind_start=ind_start)
        
        pcd_nonground_chunks, pcd_ground_chunks,\
        pcd_nonground_chunks_major_downsampling, pcd_ground_chunks_major_downsampling, \
        indices,indices_ground, center_positions, \
        center_ids, chunk_bounds, nuscenes_labels = chunk_and_downsample_point_clouds(pcd_nonground_minor, pcd_ground_minor, T_pcd, positions, 
                                                                    first_position, sampled_indices_global, chunk_size=chunk_size, 
                                                                    overlap=overlap, major_voxel_size=major_voxel_size,kitti_labels=nuscenes_labels_orig)
                                                                    
                                                                    
        
        
        
        cams = ["CAM_FRONT", "CAM_FRONT_LEFT", "CAM_FRONT_RIGHT"]
        cam_ids = [0]
        
        #out_dbscan = 'out_dbscan/'
        #if os.path.exists(out_dbscan) == True : 
        #        shutil.rmtree(out_dbscan)
        out_ncuts_cur = out_nuscenes_pred + str(SEQUENCE_NUM) + '/'
        if os.path.exists(out_ncuts_cur) == True : 
                shutil.rmtree(out_ncuts_cur)
        os.makedirs(out_ncuts_cur)
        
        
                
                
        out_instance_cur = out_nuscenes_instances + str(SEQUENCE_NUM) + '/'
        if os.path.exists(out_instance_cur) == True : 
                shutil.rmtree(out_instance_cur)
        os.makedirs(out_instance_cur)
        
        
        instances = np.hstack((nuscenes_labels_orig['instance_nonground'].reshape(-1,),nuscenes_labels_orig['instance_ground'].reshape(-1,)))
        
        patchwise_indices = indices_per_patch(T_pcd, center_positions, positions, first_position, sampled_indices_global, chunk_size)
        out_data = []
        print(len(center_ids))
        for sequence in range(len(center_ids)):
                        merged_chunk, file_name, pcd_chunk, pcd_chunk_ground, inliers, inliers_ground = ncuts_chunk(dataset, indices, pcd_nonground_chunks, 
                                pcd_ground_chunks, pcd_nonground_chunks_major_downsampling, pcd_nonground_minor, T_pcd, center_positions, center_ids,
                                positions, first_position, sampled_indices_global,
                                chunk_size=chunk_size, major_voxel_size=major_voxel_size,
                                alpha=alpha, beta=beta, gamma=gamma, theta=theta,
                                proximity_threshold=proximity_threshold, ncuts_threshold=T, cams = cams, cam_ids = cam_ids, out_folder=out_ncuts_cur, 
                                ground_mode=False, sequence=sequence, patchwise_indices=patchwise_indices, adjacent_frames_cam=(4,5), adjacent_frames_tarl=(5,5))
        
        
                        name = file_name.split('/')[-1]
                        #o3d.visualization.draw_geometries([pcd_chunk + pcd_chunk_ground])
                        o3d.io.write_point_cloud(out_ncuts_cur + name, pcd_chunk + pcd_chunk_ground , write_ascii=False, compressed=False, print_progress=False)
                        
                        seg_ground = nuscenes_labels['ground']['semantic'][sequence][inliers][inliers_ground]
                        inst_ground = nuscenes_labels['ground']['instance'][sequence][inliers][inliers_ground]
                        nuscenes_chunk_instance = color_pcd_by_labels(pcd_chunk,nuscenes_labels['nonground']['instance'][sequence].reshape(-1,),gt_labels=instances,largest=False,
                                        colors=colors)
                        nuscenes_chunk_instance_ground = color_pcd_by_labels(pcd_chunk_ground,inst_ground.reshape(-1,),gt_labels=instances,largest=False, 
                                        colors=colors)
                        print('labels',np.unique(nuscenes_labels['nonground']['instance'][sequence]))                        
                
                        #o3d.visualization.draw_geometries([nuscenes_chunk_instance + nuscenes_chunk_instance_ground])
                        print("write instance chunk")
                        o3d.io.write_point_cloud(out_instance_cur + name, nuscenes_chunk_instance + nuscenes_chunk_instance_ground, write_ascii=False, compressed=False, print_progress=False)


In [ ]:
colors = generate_random_colors_map(400,0)

def get_merge_pcds(out_folder_ncuts):
        point_clouds = []

        # List all files in the folder
        files = os.listdir(out_folder_ncuts)
        files.sort()

        # Filter files with a .pcd extension
        pcd_files = [file for file in files if file.endswith(".pcd")]
        print(pcd_files)
        # Load each point cloud and append to the list
        for pcd_file in pcd_files:
                        
                file_path = os.path.join(out_folder_ncuts, pcd_file)
                point_cloud = o3d.io.read_point_cloud(file_path)
                point_clouds.append(point_cloud)
        return point_clouds

Now we can split the point cloud into chunks based on a tbd chunk_size

In [ ]:
#out_dbscan = 'out_dbscan/'

for i in seqs: 
    print('cur seq',i)
    out_instance_cur = out_nuscenes_instances + str(i) + '/'
    out_ncuts_cur = out_nuscenes_pred + str(i) + '/'
    
    point_clouds = get_merge_pcds(out_ncuts_cur)
    if len(point_clouds) == 0 : 
        continue
    merge = merge_chunks_unite_instances(point_clouds)
    
    point_clouds_nuscenes_instances = get_merge_pcds(out_instance_cur)
    merge_nuscenes_instance = merge_unite_gt(point_clouds_nuscenes_instances)
    #o3d.visualization.draw_geometries([merge_nuscenes_instance])
    unique_colors, labels_ncuts = np.unique(np.asarray(merge.colors), axis=0, return_inverse=True)
    unique_colors, labels_nuscenes = np.unique(np.asarray(merge_nuscenes_instance.colors),axis=0, return_inverse=True)
        
    pred_instance = remove_semantics(labels_nuscenes,labels_ncuts)
    
    o3d.io.write_point_cloud(map_out_instances + "merge_part_nuscenes_instance" + str(i)  + ".pcd", merge_nuscenes_instance, write_ascii=False, compressed=False, print_progress=False)
    o3d.io.write_point_cloud(map_out_pred + "merge_part_nuscenes_ncuts" + str(i)  + ".pcd", merge, write_ascii=False, compressed=False, print_progress=False)
    o3d.io.write_point_cloud(map_out_pred + "merge_part_nuscenes_ncuts_instances_" + str(i)  + ".pcd", color_pcd_by_labels(merge,pred_instance), write_ascii=False, compressed=False, print_progress=False)
    




In [ ]:

merge_pcd_pred = o3d.geometry.PointCloud()
merge_pcd_instance = o3d.geometry.PointCloud()
merge_instances_only = o3d.geometry.PointCloud()

metrics_ncuts = Metrics(name='ncuts',min_points=50)



for i in seqs[1:5] : 
	if os.path.exists(map_out_instances + "merge_part_nuscenes_instance" + str(i)  + ".pcd") == False : 
		continue
	all_pred = o3d.io.read_point_cloud(map_out_pred + "merge_part_nuscenes_ncuts" + str(i)  + ".pcd")
	cur_gt = o3d.io.read_point_cloud(map_out_instances + "merge_part_nuscenes_instance" + str(i)  + ".pcd")
	instances_pred = o3d.io.read_point_cloud(map_out_pred + "merge_part_nuscenes_ncuts_instances_" + str(i)  + ".pcd")
	
	
	unique_colors, labels_ncuts = np.unique(np.asarray(instances_pred.colors), axis=0, return_inverse=True)
	unique_colors, labels_ncuts_all = np.unique(np.asarray(all_pred.colors), axis=0, return_inverse=True)
	unique_colors, labels_nuscenes = np.unique(np.asarray(cur_gt.colors),axis=0, return_inverse=True)
	metrics_ncuts.add_stats(labels_ncuts_all,labels_ncuts,labels_nuscenes)
	
	merge_pcd_instance += cur_gt
	merge_instances_only += instances_pred
	merge_pcd_pred +=  all_pred
	


metrics_ncuts.compute_stats_final()
	


In [ ]:
o3d.visualization.draw_geometries([merge_instances_only])